In [114]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from random import randint
from utils import get_stats, get_loaders
from dataset import NormalizedDataset, IncompleteDataset

In [115]:
white = 255
black = 0

def inverse_color(img):
    return PIL.Image.eval(img, lambda val: 255 - val)

def fix_background_color_bug(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] < colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

def remove_background(img):
    return PIL.Image.eval(img, lambda val: 0 if val < (256/2) else val)

def to_binary(img):
    return PIL.Image.eval(img, lambda val: 255 if val < (256/2) else 0)

In [125]:
binary_image = True

## Rotation, Brightness and Resizing
random_transforms = [
    transforms.RandomRotation(360, fill=black),
    transforms.RandomAffine(0, shear=15, scale=(0.8, 0.95), translate=(.03, .03)),
]

format_transforms = [
    transforms.Resize((25, 25)),
]

if not binary_image:
    random_transforms.append(transforms.ColorJitter(brightness=(0.9, 1), contrast=(0.7, 1)))
else:
    format_transforms.append(transforms.Lambda(to_binary))

all_transforms = transforms.Compose(random_transforms + format_transforms)
format_transforms = transforms.Compose(format_transforms)

## Adding Grayscale + Inverse color to operators
operators_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    all_transforms,
])

video_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    transforms.Lambda(remove_background),
    format_transforms,
])

operators_tensor_transform = transforms.Compose([
    operators_transform,
    transforms.ToTensor(),
])

video_tensor_transform = transforms.Compose([
    video_transform,
    transforms.ToTensor(),
])

In [126]:
op_video_dataset = datasets.ImageFolder(root='video_dataset/op', transform=video_tensor_transform)
datasets.ImageFolder(root='video_dataset/op', transform=video_transform)[2][0]

In [127]:
# Operators Dataset
operators_dataset = NormalizedDataset(datasets.ImageFolder(root='operators', transform=operators_tensor_transform))

# Show exemple image
datasets.ImageFolder(root='operators', transform=operators_transform)[4][0]

In [119]:
operators_mean, operators_std = get_stats(operators_dataset)
operators_mean, operators_std

(0.18310169875621796, 0.2621324956417084)

In [121]:
# Creating a dataset and loader with only the video images
op_incomplete_dataset = IncompleteDataset(op_video_dataset, operators_dataset.classes, operators_mean, operators_std)
test_loader = torch.utils.data.DataLoader(op_incomplete_dataset, batch_size=100)

In [104]:
# Training set
train_loader = torch.utils.data.DataLoader(operators_dataset, batch_size=100)
len(train_loader.dataset), len(test_loader.dataset)

(6, 4)

In [107]:
# from keras.regularizers import L1L2
# from keras.layers import Conv2D
# Creating a Net class object, which consists of 2 convolutional layers, max-pool layers and fully-connected layers
class Conv_Net(nn.Module):
    
    def __init__(self, nb_hidden=25):        
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # the first convolutional layer, which processes the input image
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set

        self.fc1 = nn.Linear(64, nb_hidden)  # the first fully-connected layer, which gets flattened max-pooled set
        self.fc2 = nn.Linear(nb_hidden, 5)  # the second fully-connected layer that outputs the result

    # Creating the forward pass
    def forward(self, x):
        
        # The first two layers
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        
#         x = nn.Dropout(p=0.5)(x)
        
        # The second two layers
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) 
        
#         x = nn.Dropout(p=0.5)(x)
        
        x = F.relu(F.max_pool2d(self.conv3(x), kernel_size=2)) 

        # Flattening the data set for fully-connected layer
        x = x.view(x.size(0), -1)
    
        # The first fully-connected layer
        x = F.relu(self.fc1(x))
        
        x = nn.Dropout(p=0.3)(x)
        
        # The second full-connected layer
        x = self.fc2(x)
        
        return x

In [109]:
%%time

model = Conv_Net()

losses = []
test_losses = []

# Defining the optimizer for GD
lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

# Train the model
nb_epochs = 50000
accs = []
best_acc = 0

for e in range(nb_epochs):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    for train_input, train_target in train_loader:

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        tmp_acc = 0
        for tensor, target in zip(output, train_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
                
        tmp_acc /= len(output)
        acc += tmp_acc 
        
    acc /= len(train_loader)
    
    should_print = e%101 == 0
    if should_print:
        print('%dth epoch, test_acc: %f' % (e+1, acc), end="")
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    for test_input, test_target in test_loader:
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        
        tmp_acc = 0
        for tensor, target in zip(output, test_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
            elif should_print:
                print(", {} != {}".format(index, target), end="")
                
        tmp_acc /= len(output)
        acc += tmp_acc 
                
    acc /= len(test_loader)
    accs.append(acc)
    
    if should_print:
        current_acc = sum(accs) / len(accs)
        if current_acc > best_acc:
            best_acc = current_acc
            torch.save(model.state_dict(), "operator_model")
            
        print(", test acc: %f" % current_acc)
        accs = []
    test_losses.append(loss)

1th epoch, test_acc: 0.166667, 3 != 0, 3 != 1, 3 != 4, test acc: 0.250000
102th epoch, test_acc: 0.666667, 0 != 1, 0 != 4, test acc: 0.309406
203th epoch, test_acc: 0.833333, 0 != 1, test acc: 0.559406
304th epoch, test_acc: 0.833333, 2 != 4, test acc: 0.680693
405th epoch, test_acc: 1.000000, 2 != 4, test acc: 0.707921
506th epoch, test_acc: 1.000000, 2 != 4, test acc: 0.717822
607th epoch, test_acc: 1.000000, test acc: 0.821782
708th epoch, test_acc: 1.000000, 2 != 4, test acc: 0.806931
809th epoch, test_acc: 1.000000, test acc: 0.819307
910th epoch, test_acc: 1.000000, 0 != 1, 2 != 4, test acc: 0.829208
1011th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.863861
1112th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.829208
1213th epoch, test_acc: 1.000000, test acc: 0.881188
1314th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.861386
1415th epoch, test_acc: 1.000000, 2 != 4, test acc: 0.898515
1516th epoch, test_acc: 1.000000, test acc: 0.888614
1617th epoch, test_acc: 1.000000, 0 

14343th epoch, test_acc: 1.000000, test acc: 0.918317
14444th epoch, test_acc: 1.000000, test acc: 0.910891
14545th epoch, test_acc: 1.000000, test acc: 0.898515
14646th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.918317
14747th epoch, test_acc: 1.000000, test acc: 0.915842
14848th epoch, test_acc: 1.000000, test acc: 0.891089
14949th epoch, test_acc: 1.000000, test acc: 0.863861
15050th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.883663
15151th epoch, test_acc: 1.000000, test acc: 0.908416
15252th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.856436
15353th epoch, test_acc: 1.000000, test acc: 0.950495
15454th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.980198
15555th epoch, test_acc: 1.000000, test acc: 0.923267
15656th epoch, test_acc: 1.000000, test acc: 0.933168
15757th epoch, test_acc: 1.000000, test acc: 0.925743
15858th epoch, test_acc: 1.000000, test acc: 0.955446
15959th epoch, test_acc: 1.000000, test acc: 0.955446
16060th epoch, test_acc: 1.000000, test acc: 0.950

29089th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.933168
29190th epoch, test_acc: 1.000000, test acc: 0.950495
29291th epoch, test_acc: 1.000000, test acc: 0.957921
29392th epoch, test_acc: 1.000000, test acc: 0.955446
29493th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.965347
29594th epoch, test_acc: 1.000000, test acc: 0.960396
29695th epoch, test_acc: 1.000000, test acc: 0.948020
29796th epoch, test_acc: 1.000000, 0 != 1, test acc: 0.962871
29897th epoch, test_acc: 1.000000, test acc: 0.915842
29998th epoch, test_acc: 1.000000, test acc: 0.891089
CPU times: user 24min 36s, sys: 2min 57s, total: 27min 33s
Wall time: 27min 41s


In [91]:
model = Conv_Net()
model.load_state_dict(torch.load("operator_model"))
criterion = nn.CrossEntropyLoss()

test_loader = torch.utils.data.DataLoader(op_incomplete_dataset, batch_size=100)
acc = 0
for test_input, test_target in test_loader:

    output = model(test_input)
    loss = criterion(output, test_target) 

    tmp_acc = 0
    for tensor, target in zip(output, test_target):
        _, index = tensor.max(0)
        if index == target:
            tmp_acc += 1
        elif True:
            print(", {} != {}".format(index, target), end="")

    tmp_acc /= len(output)
    acc += tmp_acc 

acc /= len(test_loader)


In [93]:
acc

1.0